# Python & AWS (S3 / Redshift)

### Install PostgreSQL database adapter

In [ ]:
%%!
pip install psycopg2-binary

### Import moddules

In [ ]:
import configparser
import psycopg2
import pandas as pd

In [ ]:
config = configparser.ConfigParser()
config.read('data_files/dwh.cfg')

### Create Redshift connection

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

### Get S3 connection parameters

In [ ]:
DB_REGION       = config.get('REGION', 'DB_REGION')
IAM_ARM         = config.get('IAM_ROLE', 'ARN')
S3_LOG_JDATA    = config.get('S3', 'LOG_DATA')
S3_LOG_JPATH    = config.get('S3', 'LOG_JSONPATH')

### Create a table

In [ ]:
staging_events_table_create= ("""CREATE TABLE IF NOT EXISTS public.staging_events(
                        artist          VARCHAR,
                        auth            VARCHAR,
                        firstName       VARCHAR,
                        gender          VARCHAR,
                        itemInSession   INT,
                        lastName        VARCHAR,
                        length          FLOAT,
                        level           VARCHAR,
                        location        VARCHAR,
                        method          VARCHAR,
                        page            VARCHAR,
                        registration    VARCHAR,
                        sessionId       INT,
                        song            VARCHAR,
                        status          INT,
                        ts              VARCHAR,
                        userAgent       VARCHAR,
                        userId          INT
                        )""")

In [ ]:
cur.execute(staging_events_table_create)
conn.commit()

### Load data to Redshift table

In [ ]:
staging_events_copy = ("""
    copy public.staging_events
    from {}
    credentials {}
    region {}
    json {}
""").format(S3_LOG_JDATA, IAM_ARM, DB_REGION, S3_LOG_JPATH)

In [ ]:
cur.execute(staging_events_copy)
conn.commit()

### Query to Redshift table

In [ ]:
cur.execute("""SELECT * from public.staging_events""")
conn.commit()

In [ ]:
data_frame = pd.read_sql_query('SELECT * FROM public.staging_events;', conn)

In [ ]:
data_frame.head(5)

In [ ]:
conn.close()